# Unsupervised Learning - Clustering Analysis of Eloquence LOG File

Attached are two data sets:

1) QuoteData Spreadsheet (CRM data) outlining the quotes and quote lines with products being configured

2) Engine Logs for Eloquence (10 application logs file).

The candidate can use any tool of his/her choice or liking on their own laptop or ours and mine/profile the data and present insights on the data for both CPQ and Eloquence (both are proprietary software products) Specific areas could be:

1) the type of Components Used, Processing Time, Documents Processed, Type of File, etc. for the Eloquence data and

2) product and pricing on quote and quote lines; pricing related to account, contact, product for the CPQ related Quote Data

In summary, we want the candidate to look at the two different sets of data and provide us with meaningful data insights and whatever other useful information that we could glean from the data. Though some may view the attached case study as preparation for a data reporting role, it is not. This case study is just one of the small tools (data analysis 101) that we utilize in determining the right candidate for the role. This is, in fact, a true data analyst/scientist role and will encompass a mix of strategy, advisory and implementation (though there will be business intelligence and analytics reporting for all cloud products as part of the job profile as well).

Since the business mentions profiling, this seems to be a Clustering Analysis based on Unsupervised Learning Problem, and as there is no Target Variable present, this reinforces the Clustering Analysis Approach.

1) Data dictionary not provided?

2) The clusters will be formed based on the variables given in the business problem. ex.: Components Used from the Eloquence log file

3) Data Standardization: convert prices of the products into percentages, so as to compare different products based on percent price.

4) Scree plot: x= centers (centroids) y= WCSS (Within Cluster Sum of Square); then identify the bend or curve in Scree Plot

5) Dimensionality Reduction PCA & LCA


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
# Run multiple commands and get multiple outputs within a single cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
logdf = pd.read_fwf('Engine_main_01.log.txt')
logdf.head()

,2015-05-22,"15:13:36,003",DEBUG,[Engine_main_01],[pool-2-thread-144],[cincom.eloquence.Format.reader.CSSAXFormatterHandler] imageParameters.getImagesFolder() = C:\Cincom Eloquence\EngineServer\Instances\EloqInstance01\images,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-05-22,"15:13:36,009",DEBUG,[Engine_main_01],[pool-2-thread-144],[cincom.eloquence.Format.reader.CSSAXFormatter...,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-05-22,"15:13:36,011",DEBUG,[Engine_main_01],[pool-2-thread-144],[cincom.eloquence.Format.reader.CSSAXFormatter...,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
logdf.shape

(64258, 11)

In [7]:
dfComp = []
with open("Engine_main_01.log.txt", 'r') as f:
    for line in f:
        if "Start loading component : " in line:
            line = line.split('Start loading component :')
            line = line[1]
            line = line.strip("\n',")
            dfComp.append(line)
dfComp = pd.DataFrame({'Component_Name':dfComp})

dfComp

,Component_Name
0,LP_Fixed_NHIC_Corr_Address_Block
1,FIA_30_Day_American_Legend_III_and_LIB_6_8_Pa...
2,MarketName_Legend_III_6_8_CP
3,FIA_30_Day_Legend_III_Strategy_Table_CP
4,LEGEND_III_LIB_6_8_CUSTOM_10_FNAME_FUNDNAMES_CP
5,FIA_30_Day_American_Legend_III_and_LIB_6_8_Pa...
6,Correspondence_Footer_CP
7,LEGEND_III_LIB_6_8_CUSTOM_10_FNAME_FUNDNAMES_CP
8,LP_Fixed_NHIC_30_Day_Legend_III_LIB_6_8_Secti...
9,MarketName_Legend_III_6_8_CP


In [89]:
dfComp.head()
dfComp.Component_Name.unique()

,Component_Name
0,LP_Fixed_NHIC_Corr_Address_Block
1,FIA_30_Day_American_Legend_III_and_LIB_6_8_Pa...
2,MarketName_Legend_III_6_8_CP
3,FIA_30_Day_Legend_III_Strategy_Table_CP
4,LEGEND_III_LIB_6_8_CUSTOM_10_FNAME_FUNDNAMES_CP


array([' LP_Fixed_NHIC_Corr_Address_Block',
       ' FIA_30_Day_American_Legend_III_and_LIB_6_8_Part_1_CP',
       ' MarketName_Legend_III_6_8_CP',
       ' FIA_30_Day_Legend_III_Strategy_Table_CP',
       ' LEGEND_III_LIB_6_8_CUSTOM_10_FNAME_FUNDNAMES_CP',
       ' FIA_30_Day_American_Legend_III_and_LIB_6_8_Part_3_CP',
       ' Correspondence_Footer_CP',
       ' LP_Fixed_NHIC_30_Day_Legend_III_LIB_6_8_Section_Form_Part_1',
       ' FIA_30_Day_Legend_III_LIB_6_8_1yr_18M_PtoP_Section_Form',
       ' FIA_30_Day_Legend_III_LIB_6_8_Gold_Section_Form',
       ' FIA_30_Day_Legend_III_LIB_6_8_Monthly_Sum_Section_Form',
       ' LP_Fixed_NHIC_30_Day_Selection_Legend_III_LIB_6_8_Form_Table',
       ' FIA_30_Day_Selection_Form_Legend_III_LIB_6_8_End',
       ' LP_Fixed_NHIC_Fia_30_Day_Renewal_Part_1',
       ' Company_User_Defined_CP', ' MarketName_CP',
       ' LP_Fixed_NHIC_FIA_30_Day_Renewal_032408_All_Other_Products_Table',
       ' LP_Fixed_NHIC_Correspondence_User_Defined_Variables_FundNa

In [92]:
dfComp = dfComp.drop_duplicates()

In [93]:
dfComp.to_csv('Component_Name.csv', sep= ',', encoding='utf8', index=False)